# News Modeling

Topic modeling involves **extracting features from document terms** and using
mathematical structures and frameworks like matrix factorization and SVD to generate **clusters or groups of terms** that are distinguishable from each other and these clusters of words form topics or concepts

Topic modeling is a method for **unsupervised classification** of documents, similar to clustering on numeric data

These concepts can be used to interpret the main **themes** of a corpus and also make **semantic connections among words that co-occur together** frequently in various documents

Topic modeling can help in the following areas:
- discovering the **hidden themes** in the collection
- **classifying** the documents into the discovered themes
- using the classification to **organize/summarize/search** the documents

Frameworks and algorithms to build topic models:
- Latent semantic indexing
- Latent Dirichlet allocation
- Non-negative matrix factorization

## Latent Dirichlet Allocation (LDA)
The latent Dirichlet allocation (LDA) technique is a **generative probabilistic model** where each **document is assumed to have a combination of topics** similar to a probabilistic latent semantic indexing model

In simple words, the idea behind LDA is that of two folds:
- each **document** can be described by a **distribution of topics**
- each **topic** can be described by a **distribution of words**

### LDA Algorithm

- 1. For each document, **randomly initialize each word to one of the K topics** (k is chosen beforehand)
- 2. For each document D, go through each word w and compute:
    - **P(T |D)** , which is a proportion of words in D assigned to topic T
    - **P(W |T )** , which is a proportion of assignments to topic T over all documents having the word W
- **Reassign word W with topic T** with probability P(T |D)´ P(W |T ) considering all other words and their topic assignments

![LDA](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/LDA.png)

### Steps
- Install the necessary library
- Import the necessary libraries
- Download the dataset
- Load the dataset
- Pre-process the dataset
    - Stop words removal
    - Email removal
    - Non-alphabetic words removal
    - Tokenize
    - Lowercase
    - BiGrams & TriGrams
    - Lemmatization
- Create a dictionary for the document
- Filter low frequency words
- Create an Index to word dictionary
- Train the Topic Model
- Predict on the dataset
- Evaluate the Topic Model
    - Model Perplexity
    - Topic Coherence
- Visualize the topics

### Install the necessary library

In [1]:
#! pip install pyLDAvis gensim spacy

### Import the libraries

In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from pprint import pprint
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import gensim
import wget

### Download the dataset
Dataset: https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json

#### 20-Newsgroups dataset
- 11K newsgroups posts
- 20 news topics

In [2]:
wget.download('https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json')

'newsgroups (2).json'

### Load the dataset

In [2]:
df = pd.read_json('newsgroups.json')
df.head()

,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [3]:
data = df.content.values.tolist()

### Preprocess the data

### Email Removal

In [4]:
import re
data = [re.sub('S*@S*s?', '', sent) for sent in data]

### Newline Removal

In [5]:
data = [re.sub('s+', ' ', sent) for sent in data]

### Single Quotes Removal

In [6]:
data = [re.sub("'", "", sent) for sent in data]

In [9]:
data

['From: lerx twam.umd.edu (where  my thing)\nSubject: WHAT car i  thi !?\nNntp-Po ting-Ho t: rac3.wam.umd.edu\nOrganization: Univer ity of Maryland, College Park\nLine : 15\n\n I wa  wondering if anyone out there could enlighten me on thi  car I  aw\nthe other day. It wa  a 2-door  port  car, looked to be from the late 60 /\nearly 70 . It wa  called a Bricklin. The door  were really  mall. In addition,\nthe front bumper wa   eparate from the re t of the body. Thi  i  \nall I know. If anyone can tellme a model name, engine  pec , year \nof production, where thi  car i  made, hi tory, or whatever info you\nhave on thi  funky looking car, plea e e-mail.\n\nThank ,\n- IL\n   ---- brought to you by your neighborhood Lerx t ----\n\n\n\n\n',
 'From: guykuocar on.u.wa hington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock report \nKeyword : SI,acceleration,clock,upgrade\nArticle-I.D.:  helley.1qvfo9INNc3 \nOrganization: Univer ity of Wa hington\nLine : 11\

### Tokenize
- Create **sent_to_words()** 
    - Use **gensim.utils.simple_preprocess**
    - Use **generator** instead of an usual function

In [7]:
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [8]:
data_words = list(sent_to_words(data))
print(data_words[:1][0][:25])

['from', 'lerx', 'twam', 'umd', 'edu', 'where', 'my', 'thing', 'subject', 'what', 'car', 'thi', 'nntp', 'po', 'ting', 'ho', 'rac', 'wam', 'umd', 'edu', 'organization', 'univer', 'ity', 'of', 'maryland']


In [9]:
print(data_words[:1][0][:100])

['from', 'lerx', 'twam', 'umd', 'edu', 'where', 'my', 'thing', 'subject', 'what', 'car', 'thi', 'nntp', 'po', 'ting', 'ho', 'rac', 'wam', 'umd', 'edu', 'organization', 'univer', 'ity', 'of', 'maryland', 'college', 'park', 'line', 'wa', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'thi', 'car', 'aw', 'the', 'other', 'day', 'it', 'wa', 'door', 'port', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'wa', 'called', 'bricklin', 'the', 'door', 'were', 'really', 'mall', 'in', 'addition', 'the', 'front', 'bumper', 'wa', 'eparate', 'from', 'the', 're', 'of', 'the', 'body', 'thi', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'pec', 'year', 'of', 'production', 'where', 'thi', 'car', 'made', 'hi', 'tory', 'or', 'whatever']


### Stop words Removal
- Extend the stop words corpus with the following words
    - from
    - subject
    - re
    - edu
    - use

In [10]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(len(stop_words))
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
print(len(stop_words))

179
184


#### remove_stopwords( )

In [11]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [12]:
filter_tokens= remove_stopwords(data_words)

In [13]:
print(filter_tokens)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Bigrams
- Use **gensim.models.Phrases**
- 100 as threshold

In [14]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 

In [15]:
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

#### make_bigrams( )

In [16]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [17]:
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [19]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

In [20]:
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

### Lemmatization
- Use spacy
    - Download spacy en model (if you have not done that before)
    - Load the spacy model

In [21]:
import spacy


In [22]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

#### lemmatizaton( )

In [23]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
data_words_bigrams = make_bigrams(filter_tokens)

In [25]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [26]:
print(data_lemmatized[:1])

[['twam', 'umd', 'thing', 'car', 'thi', 'univer_ity', 'park', 'line', 'wa', 'wonder', 'enlighten', 'thi', 'car', 'day', 'wa', 'door', 'port', 'car', 'look', 'late', 'early', 'wa', 'call', 'really', 'mall', 'addition', 'front_bumper', 'wa', 'eparate', 'body', 'thi', 'know', 'tellme', 'model', 'name', 'engine', 'pec', 'year', 'production', 'car', 'make', 'hi', 'tory', 'info', 'thi', 'funky', 'look', 'car', 'plea', 'mail', 'thank', 'bring', 'neighborhood', 'lerx']]


### Create a Dictionary

In [27]:
dictionary = corpora.Dictionary(data_lemmatized)

### Create Corpus

In [28]:
texts = data_lemmatized

### Filter low-frequency words

In [29]:
dictionary.filter_extremes(no_below=10, no_above=0.5)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [30]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 4), (30, 1), (31, 1), (32, 1), (33, 1), (34, 4), (35, 1), (36, 1)]]


In [31]:
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 4), (30, 1), (31, 1), (32, 1), (33, 1), (34, 4), (35, 1), (36, 1)]]


### Create Index 2 word dictionary

In [37]:
##temp = dictionary[0]  # This is only to "load" the dictionary.
##id2word = dictionary.id2token

In [39]:
##print(id2word)

{0: 'addition', 1: 'body', 2: 'bring', 3: 'call', 4: 'car', 5: 'day', 6: 'door', 7: 'early', 8: 'engine', 9: 'enlighten', 10: 'eparate', 11: 'hi', 12: 'info', 13: 'know', 14: 'late', 15: 'look', 16: 'mail', 17: 'make', 18: 'mall', 19: 'model', 20: 'name', 21: 'neighborhood', 22: 'park', 23: 'pec', 24: 'plea', 25: 'port', 26: 'production', 27: 'really', 28: 'thank', 29: 'thi', 30: 'thing', 31: 'tory', 32: 'umd', 33: 'univer_ity', 34: 'wa', 35: 'wonder', 36: 'year', 37: 'acceleration', 38: 'add', 39: 'age', 40: 'article', 41: 'attain', 42: 'brave', 43: 'brief', 44: 'cillator', 45: 'clock', 46: 'cpu', 47: 'detail', 48: 'di', 49: 'do', 50: 'end', 51: 'experience', 52: 'fair', 53: 'final', 54: 'floppie', 55: 'floppy', 56: 'functionality', 57: 'hare', 58: 'have', 59: 'heat_ink', 60: 'helley', 61: 'hington', 62: 'hour', 63: 'network', 64: 'next', 65: 'nntp', 66: 'number', 67: 'oul', 68: 'pecially', 69: 'peed', 70: 'poll', 71: 'procedure', 72: 'rate', 73: 'reque', 74: 'si', 75: 'summary', 76: 

In [47]:
##id2word1 = corpora.Dictionary(data_lemmatized)

In [48]:
##print(id2word1)

Dictionary(53332 unique tokens: ['addition', 'body', 'bring', 'call', 'car']...)


### Build a News Topic Model

#### LdaModel
- **num_topics** : this is the number of topics you need to define beforehand
- **chunksize** : the number of documents to be used in each training chunk
- **alpha** : this is the hyperparameters that affect the sparsity of the topics
- **passess** : total number of training assess

In [32]:
ldamodel = LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)

### Print the Keyword in the 10 topics

In [33]:
pprint(ldamodel.top_topics(corpus,topn=10))

[([(0.012810896, 'thi'),
   (0.0115286615, 'people'),
   (0.010026059, 'think'),
   (0.009503566, 'doe'),
   (0.0093656145, 'wa'),
   (0.00863666, 'do'),
   (0.008176044, 'know'),
   (0.008107465, 'believe'),
   (0.0078029595, 'make'),
   (0.006264751, 'ay')],
  -1.0176838129077537),
 ([(0.099568225, 'com'),
   (0.032726973, 'article'),
   (0.012442106, 'do'),
   (0.011879733, 'nntp'),
   (0.010894091, 'apr'),
   (0.009554852, 'get'),
   (0.008068419, 'reply'),
   (0.007860748, 'm'),
   (0.0073530343, 'thi'),
   (0.0071628736, 'wa')],
  -1.096103660888277),
 ([(0.03469147, 'wa'),
   (0.022597427, 'do'),
   (0.021545252, 'go'),
   (0.017434081, 'get'),
   (0.014807346, 'know'),
   (0.013328847, 'think'),
   (0.012039258, 'people'),
   (0.011525532, 'come'),
   (0.010903025, 'aid'),
   (0.0093675265, 'time')],
  -1.1122037392104327),
 ([(0.020628063, 'game'),
   (0.01919034, 'team'),
   (0.01563345, 'play'),
   (0.014578514, 'year'),
   (0.013204255, 'wa'),
   (0.012532232, 'player'),
  

In [34]:
pprint(ldamodel.print_topics())
doc_lda = ldamodel[corpus]

[(0,
  '0.035*"wa" + 0.023*"do" + 0.022*"go" + 0.017*"get" + 0.015*"know" + '
  '0.013*"think" + 0.012*"people" + 0.012*"come" + 0.011*"aid" + 0.009*"time"'),
 (1,
  '0.013*"thi" + 0.012*"people" + 0.010*"think" + 0.010*"doe" + 0.009*"wa" + '
  '0.009*"do" + 0.008*"know" + 0.008*"believe" + 0.008*"make" + 0.006*"ay"'),
 (2,
  '0.015*"ion" + 0.012*"pace" + 0.009*"new" + 0.009*"information" + '
  '0.007*"earch" + 0.007*"datum" + 0.006*"space" + 0.006*"include" + '
  '0.006*"available" + 0.005*"project"'),
 (3,
  '0.161*"_" + 0.037*"m" + 0.022*"ei" + 0.017*"ax" + 0.016*"helmet" + '
  '0.015*"rlk" + 0.011*"dy" + 0.011*"bj" + 0.009*"air" + 0.009*"sp"'),
 (4,
  '0.030*"drive" + 0.022*"univer_ity" + 0.018*"thank" + 0.015*"plea" + '
  '0.014*"nntp" + 0.014*"card" + 0.013*"mail" + 0.013*"driver" + 0.012*"know" '
  '+ 0.011*"new"'),
 (5,
  '0.100*"com" + 0.033*"article" + 0.012*"do" + 0.012*"nntp" + 0.011*"apr" + '
  '0.010*"get" + 0.008*"reply" + 0.008*"m" + 0.007*"thi" + 0.007*"wa"'),
 (6,
  '

## Evaluation of Topic Models
- Model Perplexity
- Topic Coherence

### Model Perplexity

Model perplexity is a measurement of **how well** a **probability distribution** or probability model **predicts a sample**

### Topic Coherence
Topic Coherence measures score a single topic by measuring the **degree of semantic similarity** between **high scoring words** in the topic.

In [35]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=data_lemmatized,corpus=corpus, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4832741176584463


In [36]:
from gensim.models import LsiModel
lsimodel = LsiModel(corpus, num_topics=10, id2word = dictionary)
pprint(lsimodel.print_topics(num_topics=10, num_words=10))

[(0,
  '0.315*"wa" + 0.249*"do" + 0.223*"thi" + 0.200*"file" + 0.177*"go" + '
  '0.166*"get" + 0.162*"people" + 0.162*"know" + 0.135*"make" + 0.121*"think"'),
 (1,
  '-0.525*"file" + 0.319*"wa" + 0.194*"go" + -0.187*"image" + 0.186*"do" + '
  '-0.155*"program" + 0.129*"know" + 0.129*"people" + 0.128*"aid" + '
  '-0.119*"available"'),
 (2,
  '-0.975*"_" + -0.093*"m" + -0.085*"rlk" + -0.070*"ei" + -0.054*"qs" + '
  '-0.053*"sp" + -0.051*"sy" + -0.044*"dy" + -0.042*"sj" + -0.039*"r"'),
 (3,
  '0.638*"file" + 0.203*"wa" + 0.165*"entry" + -0.130*"com" + -0.125*"thi" + '
  '0.115*"gun" + -0.115*"available" + 0.111*"go" + 0.105*"output" + '
  '-0.100*"image"'),
 (4,
  '-0.507*"image" + -0.289*"jpeg" + -0.173*"color" + 0.169*"new" + -0.153*"do" '
  '+ -0.146*"format" + 0.142*"privacy" + 0.122*"thi" + 0.121*"internet" + '
  '-0.120*"ver_ion"'),
 (5,
  '-0.368*"wire" + 0.217*"wa" + 0.170*"internet" + -0.159*"thi" + '
  '0.159*"privacy" + -0.158*"ground" + -0.155*"entry" + 0.148*"com" + '
  '0.14

### Visualize the Topic Model
- Use **pyLDAvis**
    - designed to help users **interpret the topics** in a topic model that has been fit to a corpus of text data
    - extracts information from a fitted LDA topic model to inform an interactive web-based visualization

In [37]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis

/Users/harshwardhanbabel/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [38]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)

/Users/harshwardhanbabel/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
LDAvis_prepared

/Users/harshwardhanbabel/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.080574  0.014416       1        1  17.895915
0      0.082012  0.153399       2        1  11.114863
10     0.047792 -0.023662       3        1   9.877528
5      0.070713  0.128268       4        1   8.341188
12     0.078540  0.108746       5        1   7.980818
9      0.063832  0.119932       6        1   7.014956
2     -0.028687 -0.167828       7        1   6.174658
14     0.043230 -0.127801       8        1   5.775010
4      0.012522  0.092952       9        1   5.412242
7      0.041115 -0.126612      10        1   5.148224
13     0.060363  0.140837      11        1   4.533378
6     -0.027043 -0.114063      12        1   3.811620
8      0.027863 -0.138895      13        1   3.395134
11    -0.073343 -0.171032      14        1   2.508036
3     -0.479484  0.111340      15        1   1.016429, topic_info=      Term          Freq         Total Category  logprob  loglift
176    com  11680.000000  11680.000000  Default  30.0000  30.0000
807      _   1804.000000   1804.000000  Default  29.0000  29.0000
123      m   4151.000000   4151.000000  Default  28.0000  28.0000
618    key   2001.000000   2001.000000  Default  27.0000  27.0000
329   file   2833.000000   2833.000000  Default  26.0000  26.0000
...    ...           ...           ...      ...      ...      ...
1105    au     63.082163    449.073631  Topic15  -5.1752   2.6261
34      wa     81.290710  10540.490507  Topic15  -4.9216  -0.2761
2692    ri     57.540870    427.780098  Topic15  -5.2671   2.5828
372   part     55.572689   1753.307797  Topic15  -5.3019   1.1373
107     go     56.499786   5220.262417  Topic15  -5.2854   0.0628

[1116 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
807      14  0.002771          _
807      15  0.996585          _
462       1  0.977460   ab_olute
462       7  0.017990   ab_olute
81        3  0.293403       acce
...     ...       ...        ...
36       12  0.055556       year
36       14  0.044327       year
396      14  0.985100  ynchronou
4905      4  0.974449    yndrome
5490      8  0.989756      zioni

[3245 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 11, 6, 13, 10, 3, 15, 5, 8, 14, 7, 9, 12, 4])